In [4]:
import numpy as np
from stable_baselines3 import PPO
from myosuite.utils import gym as myo_gym

from config import Config
from hrl_utils import build_worker_obs


In [5]:
cfg = Config()

env = myo_gym.make(cfg.env_id)
env.reset()

worker = PPO.load("./logs/exp6/worker/worker.zip")  # adjust path

obs_dict = env.obs_dict


/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.obs_dict to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.obs_dict` for environment variables or `env.get_wrapper_attr('obs_dict')` that will search the reminding wrappers.
  logger.warn(


In [11]:
obs_dict['touching_info']

array([0, 0, 0, 0, 0, 0])